In [1]:
import cv2, os, time
import numpy as np
import tensorflow as tf


/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_dir = '../dataset/animate_tfrecord/'
save_dir = '../data/animate_label_twingan/'
n_files = 128
split_size = 5000

In [3]:
# Retrieve Tags

n_tag = 0
tag_map = {}

with open(os.path.join(data_dir, 'animate_face_tag_list.txt'), 'r') as f:
    lines = f.readlines()
    n_tag = len(lines)
    for i in range(n_tag):
        tag_map[lines[i].split('\t')[1]] = i

print (tag_map)

{'long_hair': 0, 'short_hair': 1, 'blush': 2, 'smile': 3, 'blonde_hair': 4, 'blue_eyes': 5, 'brown_hair': 6, 'open_mouth': 7, 'hat': 8, 'red_eyes': 9, 'black_hair': 10, 'ribbon': 11, 'twintails': 12, 'brown_eyes': 13, 'blue_hair': 14, 'green_eyes': 15, 'purple_eyes': 16, 'pink_hair': 17, 'purple_hair': 18, 'closed_eyes': 19, 'ponytail': 20, 'green_hair': 21, 'simple_background': 22, 'red_hair': 23, 'yellow_eyes': 24, 'very_long_hair': 25, 'glasses': 26, 'silver_hair': 27, 'one_eye_closed': 28, 'ahoge': 29, 'male': 30, 'white_hair': 31, 'earrings': 32, 'tongue': 33, 'pink_eyes': 34, 'aqua_eyes': 35, 'black_eyes': 36, 'orange_hair': 37, 'aqua_hair': 38, 'grin': 39, 'multicolored_hair': 40, 'dark_skin': 41, 'tongue_out': 42, 'grey_hair': 43, 'drill_hair': 44, 'short_twintails': 45, 'orange_eyes': 46, 'facial_hair': 47, 'grey_eyes': 48, 'two-tone_hair': 49, 'tan': 50}


In [4]:
def _parse_function(example_proto):
    keys_to_features = {
        'image/encoded': tf.FixedLenFeature(
            (), tf.string, default_value=''),
        'image/format': tf.FixedLenFeature(
            (), tf.string, default_value='jpeg'),
        'image/class/label': tf.VarLenFeature(
            dtype=tf.int64),
        'image/class/text': tf.FixedLenFeature(
            [], dtype=tf.string, default_value=''),
        'image/filename': tf.FixedLenFeature(
            [], dtype=tf.string, default_value=''),
    }
    parsed_features = tf.parse_example(example_proto, keys_to_features)
    return parsed_features['image/encoded'], parsed_features['image/class/text']

In [5]:
def _transfer_label(label, n_tag, tag_map):
    one_hot = [0. for i in range(n_tag)]
    labels = label.split(', ')   
    
    for i in range(len(labels)):
        one_hot[tag_map[labels[i]]] = 1.
    return one_hot

In [6]:
dataset = None


filenames = [data_dir + 'train-{}-of-00128'.format(str(i).zfill(5)) for i in range(n_files)]
dataset = tf.data.TFRecordDataset(filenames)
    
sess = tf.Session()


In [7]:
batch_dataset = dataset.batch(100)
iterator = batch_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

data_cnt = 0
file_cnt = 0

data = []
label = []

# sum_rgb = [0., 0., 0.]
mean_rgb = np.array([172.83672573366928, 175.73611140584845, 193.70488181437796])
mean_rgb = mean_rgb / 255.

while True:
    try:
        batch_data = sess.run(next_element)
        metadata = _parse_function(batch_data)

        image_data = metadata[0].eval(session = sess)
        label_data = metadata[1].eval(session = sess)
        
        for i in range(len(image_data)):
            nparr = np.fromstring(image_data[i], np.uint8)
            img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            img_np = cv2.resize(img_np, (96, 96))
            
            img_np = img_np / 255 - mean_rgb
            
            
            # for j in range(3):
            #    sum_rgb[j] += img_np[:, :, j].mean()
            
            data.append(img_np)

            label.append(_transfer_label(label_data[i].decode('utf-8'), n_tag, tag_map))

            data_cnt += 1

    except:
        break
    
    if len(data) == split_size:
        data = np.array(data)
        label = np.array(label)
        print (data.shape)
        print (np.max(data), np.min(data))
        np.save(save_dir + '{}_image.npy'.format(str(file_cnt).zfill(3)), data)
        np.save(save_dir + '{}_label.npy'.format(str(file_cnt).zfill(3)), label)
        file_cnt += 1
        data = []
        label = []

if len(data) > 0:
    data = np.array(data)
    label = np.array(label)
    print (data.shape)
    print (label.shape)
    np.save(save_dir + '{}_image.npy'.format(str(file_cnt).zfill(3)), data)
    np.save(save_dir + '{}_label.npy'.format(str(file_cnt).zfill(3)), label)

    
#for i in range(3):
#    sum_rgb[i] = sum_rgb[i] / data_cnt

/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


(5000, 96, 96, 3)
0.32220891869149304 -0.7596269875073646
(5000, 96, 96, 3)
0.32220891869149304 -0.7596269875073646
(5000, 96, 96, 3)
0.32220891869149304 -0.7596269875073646
(5000, 96, 96, 3)
0.32220891869149304 -0.7596269875073646
(5000, 96, 96, 3)
0.32220891869149304 -0.7596269875073646
(2243, 96, 96, 3)
(2243, 51)


In [10]:
print (sum_rgb)

[172.83672573366928, 175.73611140584845, 193.70488181437796]
